In [ ]:
#pip install djitellopy opencv-python numpy

# Velocity based takeoff

In [ ]:
from djitellopy import Tello
import cv2
import numpy as np
import time
import logging

# Suppress INFO and WARNING messages from djitellopy library
logging.getLogger('djitellopy').setLevel(logging.ERROR)

'''
World building
'''
#####################################
# Initialization of coordinates
#####################################
# Coordinate system:
#   x: positive = forward, negative = backward
#   y: positive = left, negative = right
#   z: positive = up, negative = down
# Initial starting position of ego drone (at the center of the picnic blanket)
ego_x = 0
ego_y = 0
ego_z = 0

# State history: stores full state (x, y, z) at each time step
state_history = []  # List of tuples: [(x, y, z), ...]

# Flags to track alternative paths (reset at start of each iteration)
alternative_path_x_taken = False  # Prohibits x-direction movement temporarily
alternative_path_y_taken = False  # Prohibits y-direction movement temporarily

#####################################
# Define borders
#####################################
# Define bounding box around initial starting point (200 x 200)
# Box extends 100 units in each direction from center
# BOX_SIZE = 200 # random estimate rn for the bounding box of the ODD, will test based on picnic blanket
# BOX_HALF = BOX_SIZE / 2  # 100 units
# box_x_min = ego_x - BOX_HALF
# box_x_max = ego_x + BOX_HALF
# box_y_min = ego_y - BOX_HALF
# box_y_max = ego_y + BOX_HALF

#####################################
# Adversary position (stationary)
#####################################
adversary_active = True  # Set to True to enable adversary avoidance

# Adversary position (will be set after takeoff to same z-height)
adversary_x = 40 #20  # Fixed x position of adversary
adversary_y = 0   # Fixed y position of adversary
adversary_z = None  # Will be set to takeoff height after takeoff

# Threshold distance for collision avoidance (squared distance: x^2 + y^2)
avoidance_radius = 30  # 10 #cm threshold (reduced from 20 for looser avoidance)
adversary_threshold = avoidance_radius**2  # 10^2 = 100 (10 cm threshold)

#####################################
# Goal tracking
#####################################
goal_index = 0
goals = [
    # (20, 0),   # Goal 1: Forward
    # (20, 20),  # Goal 2: Forward and left
    # (20, -20)   # Goal 3: Forward and Right Two Steps

    # (0, 20),   # Goal 1: Left
    # (20, 20),  # Goal 2: Left, then move Forward
    # (20, -20)   # Goal 3: Left, Forward, then Right Two Steps

    (0, 80),   # Goal 1: Left
    (40, 80),  # Goal 2: Left, then move Forward
    (40, -80)   # Goal 3: Left, Forward, then Right Two Steps
]

'''
Drone building
'''
#####################################
# Initialization of motion params
#####################################
upward_initialization = 20 #20 cm is the minimum
move_step = 40

#####################################
# Initialization of drone
#####################################
tello = Tello()
#print("Connecting to Tello...")
tello.connect()
print(f"Battery level: {tello.get_battery()}%")
tello.streamon()

print("Press 't' to takeoff, 'q' to land and quit.")
drone_in_air = False

#####################################
# Definitions
#####################################
def compute_switching_filter(new_x, new_y, adversary_x, adversary_y):

    dist_to_adversary_sq = (new_x - adversary_x)**2 + (new_y - adversary_y)**2
    boolean_filter = dist_to_adversary_sq < adversary_threshold
    return boolean_filter  # returns true if distance to adversary is less than threshold

def is_goal_blocked(goal_x, goal_y, adversary_x, adversary_y):

    dist_to_adversary_sq = (goal_x - adversary_x)**2 + (goal_y - adversary_y)**2
    return dist_to_adversary_sq < adversary_threshold

def find_alternative_path(current_x, current_y, goal_x, goal_y, adversary_x, adversary_y, blocked_direction='x'):


    dx = goal_x - current_x  # positive = need to go forward, negative = need to go backward
    dy = goal_y - current_y  # positive = need to go left, negative = need to go right
    
    alternatives = []
    
    # If x-direction is blocked, try moving in y-direction
    if blocked_direction == 'x':
        # First try moving in the y-direction that helps reach the goal (if dy != 0)
        if dy != 0:
            # Try moving in the direction of the goal's y component
            alt_x = current_x
            alt_y = current_y + move_step if dy > 0 else current_y - move_step
            if not compute_switching_filter(alt_x, alt_y, adversary_x, adversary_y):
                alternatives.append((alt_x, alt_y))
        
        # If that doesn't work, try the opposite y direction
        if not alternatives and dy != 0:
            alt_x = current_x
            alt_y = current_y - move_step if dy > 0 else current_y + move_step
            if not compute_switching_filter(alt_x, alt_y, adversary_x, adversary_y):
                alternatives.append((alt_x, alt_y))
        
        # If that doesn't work or dy == 0, try left and right (to go around obstacle)
        if not alternatives:
            # Try left first (positive y)
            alt_x = current_x
            alt_y = current_y + move_step
            if not compute_switching_filter(alt_x, alt_y, adversary_x, adversary_y):
                alternatives.append((alt_x, alt_y))
        
        # If left doesn't work, try right
        if not alternatives:
            alt_x = current_x
            alt_y = current_y - move_step
            if not compute_switching_filter(alt_x, alt_y, adversary_x, adversary_y):
                alternatives.append((alt_x, alt_y))
    
    # If y-direction is blocked, try moving in x-direction
    elif blocked_direction == 'y':
        # First try moving in the x-direction that helps reach the goal (if dx != 0)
        if dx != 0:
            # Try moving in the direction of the goal's x component
            alt_x = current_x + move_step if dx > 0 else current_x - move_step
            alt_y = current_y
            if not compute_switching_filter(alt_x, alt_y, adversary_x, adversary_y):
                alternatives.append((alt_x, alt_y))
        
        # If that doesn't work or dx == 0, try forward and backward (to go around obstacle)
        if not alternatives:
            # Try forward first (positive x)
            alt_x = current_x + move_step
            alt_y = current_y
            if not compute_switching_filter(alt_x, alt_y, adversary_x, adversary_y):
                alternatives.append((alt_x, alt_y))
        
        # If forward doesn't work, try backward
        if not alternatives:
            alt_x = current_x - move_step
            alt_y = current_y
            if not compute_switching_filter(alt_x, alt_y, adversary_x, adversary_y):
                alternatives.append((alt_x, alt_y))
    
    # Return the first safe alternative found (prioritized by goal direction)
    if alternatives:
        return alternatives[0]
    
    return None  # No safe alternative path found






'''
Takeoff
'''
#####################################
# Drone maneuverance after takeoff
#####################################
try:
    # Record initial state before loop starts
    state_history.append((ego_x, ego_y, ego_z))
    
    while True:
        frame = tello.get_frame_read().frame
        frame = cv2.resize(frame, (720, 480))
        h, w, _ = frame.shape

        # Display the frame
        cv2.imshow("Tello Tracking Feed", frame)

        key = cv2.waitKey(1) & 0xFF

        # Controls for drone
        if key == ord('t') and not drone_in_air:
            tello.takeoff()
            drone_in_air = True
            print("Drone is now airborne and hovering.")
            tello.move_up(upward_initialization)  # move up 30 cm after takeoff
            ego_z += upward_initialization
            adversary_z = ego_z  # Set pursuer z to same height as takeoff height
            state_history.append((ego_x, ego_y, ego_z))  # Record state after movement
            print(f"Drone moved up 30 cm. Current state: ({ego_x}, {ego_y}, {ego_z})")
            print(f"Pursuer set at position: ({adversary_x}, {adversary_y}, {adversary_z})")
            if adversary_active:
                print(f"Adversary detected at ({adversary_x}, {adversary_y}), avoidance radius: {avoidance_radius} cm")

        # if key == ord('t') and not drone_in_air:
        #     tello.takeoff()
        #     time.sleep(2)  # Delay after takeoff to allow drone to stabilize
        #     drone_in_air = True
        #     print("Drone is now airborne.")
            
        #     # Gradual takeoff using velocity control
        #     target_height = 30  # cm you want to reach (minimum safe height)
        #     hover_speed = 10    # cm/s (upward velocity)
        #     steps = int(target_height / hover_speed)
            
        #     for _ in range(steps):
        #         tello.send_rc_control(0, 0, hover_speed, 0)  # (left-right, forward-back, up-down, yaw)
        #         ego_z += hover_speed
        #         state_history.append((ego_x, ego_y, ego_z))
        #         time.sleep(1)  # sleep 1 second per step

        #     # Stop upward motion and hover
        #     #tello.send_rc_control(0, 0, 0, 0)
        #     time.sleep(0.5)  # Delay after stopping motion
        #     adversary_z = ego_z
        #     print(f"Reached hover height: {ego_z} cm")
        #     if adversary_active:
        #         print(f"Adversary detected at ({adversary_x}, {adversary_y}), avoidance radius: {avoidance_radius} cm")

        


        # drone movement based on goals
        if drone_in_air and goal_index < len(goals):
            target_x, target_y = goals[goal_index]
            
            # Check if goal is blocked by adversary (switching-based control)
            if adversary_active and is_goal_blocked(target_x, target_y, adversary_x, adversary_y):
                print(f"Goal {goal_index + 1} at ({target_x}, {target_y}) is BLOCKED by adversary at ({adversary_x}, {adversary_y})")
                print(f"Skipping goal {goal_index + 1} and moving to next goal...")
                goal_index += 1
                time.sleep(0.5)
                
                # Check if we've processed all goals (no linked list - don't return to skipped goals)
                if goal_index >= len(goals):
                    print("All goals processed! Landing the drone...")
                    if drone_in_air:
                        tello.land()
                        time.sleep(3)  # Delay after landing
                        state_history.append((ego_x, ego_y, ego_z))  # Record final state
                        print("Drone has landed.")
                        print(f"Final state: ({ego_x}, {ego_y}, {ego_z})")
                        print(f"Total states recorded: {len(state_history)}")
                    break
                continue
            
            # Calculate distance to current goal
            dx = target_x - ego_x
            dy = target_y - ego_y
            distance_to_goal = np.sqrt(dx**2 + dy**2)
            
            # Threshold for considering goal reached (within 5 cm)
            goal_threshold = 5
            
            if distance_to_goal > goal_threshold:
                movement_made = False

                # Initialize flags for this movement cycle (reset at end of each iteration)
                #alternative_path_x_taken = False
                #alternative_path_y_taken = False
                print(f"Starting next movement cycle, for now alternative_path_x_taken is {alternative_path_x_taken} and alternative_path_y_taken is {alternative_path_y_taken}")

                # Move in x direction first (only if not prohibited by alternative path)
                if abs(dx) > 0 and not alternative_path_x_taken:
                    # Calculate potential new position
                    new_x = ego_x + move_step if dx > 0 else ego_x - move_step
                    new_y = ego_y  # y stays the same for x movement
                    
                    # Check if movement is safe (if adversary is active)
                    can_move_x = True
                    if adversary_active:
                        if compute_switching_filter(new_x, new_y, adversary_x, adversary_y):
                            can_move_x = False
                            print(f"CANT MOVE TOWARDS GOAL {goal_index + 1} at {goals[goal_index]} IN X DIRECTION, ADVERSARY BLOCKING")
                            # Try alternative path (x-direction is blocked, so try y-direction)
                            alt_waypoint = find_alternative_path(ego_x, ego_y, target_x, target_y, adversary_x, adversary_y, blocked_direction='x')
                            if alt_waypoint:
                                alt_x, alt_y = alt_waypoint
                                alt_dx = alt_x - ego_x
                                alt_dy = alt_y - ego_y
                                # Print and execute alternative movement
                                if abs(alt_dy) > 0:
                                    # Alternative path moves in y direction
                                    if alt_dy > 0:
                                        print(f"Taking alternative path: Moving left to ({alt_x:.1f}, {alt_y:.1f})")
                                        tello.move_left(move_step)  # positive y = left
                                        time.sleep(0.5)  # Delay after movement
                                        ego_y += move_step
                                    else:
                                        print(f"Taking alternative path: Moving right to ({alt_x:.1f}, {alt_y:.1f})")
                                        tello.move_right(move_step)  # negative y = right
                                        time.sleep(0.5)  # Delay after movement
                                        ego_y -= move_step
                                    alternative_path_y_taken = True  # Prohibit y-direction temporarily
                                elif abs(alt_dx) > 0:
                                    # Alternative path moves in x direction
                                    if alt_dx > 0:
                                        print(f"Taking alternative path: Moving forward to ({alt_x:.1f}, {alt_y:.1f})")
                                        tello.move_forward(move_step)  # positive x = forward
                                        time.sleep(0.5)  # Delay after movement
                                        ego_x += move_step
                                    else:
                                        print(f"Taking alternative path: Moving back to ({alt_x:.1f}, {alt_y:.1f})")
                                        tello.move_back(move_step)  # negative x = backward
                                        time.sleep(0.5)  # Delay after movement
                                        ego_x -= move_step
                                    alternative_path_x_taken = True  # Prohibit x-direction temporarily
                                state_history.append((ego_x, ego_y, ego_z))
                                print(f"Alternative path taken. Current position: ({ego_x:.1f}, {ego_y:.1f}); alternative_path_x_taken is {alternative_path_x_taken}, alternative_path_y_taken is {alternative_path_y_taken} ")
                                movement_made = True
                                time.sleep(0.5)
                            else:
                                print(f"No alternative path found for X movement. Trying Y direction or skipping...")
                    
                    # Execute movement if safe
                    if can_move_x and not alternative_path_x_taken:
                        if dx > 0:
                            tello.move_forward(move_step)  # positive x = forward
                            time.sleep(0.5)  # Delay after movement
                        else:
                            tello.move_back(move_step)  # negative x = backward
                            time.sleep(0.5)  # Delay after movement
                        ego_x = new_x
                        state_history.append((ego_x, ego_y, ego_z))
                        print(f"Can move x. Moving towards goal {goal_index + 1}: ({target_x}, {target_y}), Current: ({ego_x:.1f}, {ego_y:.1f})")
                        movement_made = True
                        time.sleep(0.5)

                # Then move in y direction (only if not prohibited and x movement didn't happen or was successful)
                if abs(dy) > 0 and not alternative_path_y_taken and (not movement_made or abs(dx) == 0 or alternative_path_x_taken):
                    # Calculate potential new position
                    new_x = ego_x  # x stays the same for y movement
                    new_y = ego_y + move_step if dy > 0 else ego_y - move_step
                    
                    # Check if movement is safe (if adversary is active)
                    can_move_y = True
                    if adversary_active:
                        if compute_switching_filter(new_x, new_y, adversary_x, adversary_y):
                            can_move_y = False
                            print(f"CANT MOVE TOWARDS GOAL {goal_index + 1} at {goals[goal_index]} IN Y DIRECTION, ADVERSARY BLOCKING")
                            # Try alternative path (y-direction is blocked, so try x-direction)
                            alt_waypoint = find_alternative_path(ego_x, ego_y, target_x, target_y, adversary_x, adversary_y, blocked_direction='y')
                            if alt_waypoint:
                                alt_x, alt_y = alt_waypoint
                                alt_dx = alt_x - ego_x
                                alt_dy = alt_y - ego_y
                                # Print and execute alternative movement
                                if abs(alt_dx) > 0:
                                    # Alternative path moves in x direction
                                    if alt_dx > 0:
                                        print(f"Taking alternative path: Moving forward to ({alt_x:.1f}, {alt_y:.1f})")
                                        tello.move_forward(move_step)  # positive x = forward
                                        time.sleep(0.5)  # Delay after movement
                                        ego_x += move_step
                                    else:
                                        print(f"Taking alternative path: Moving back to ({alt_x:.1f}, {alt_y:.1f})")
                                        tello.move_back(move_step)  # negative x = backward
                                        time.sleep(0.5)  # Delay after movement
                                        ego_x -= move_step
                                    alternative_path_x_taken = True  # Prohibit x-direction temporarily
                                elif abs(alt_dy) > 0:
                                    # Alternative path moves in y direction
                                    if alt_dy > 0:
                                        print(f"Taking alternative path: Moving left to ({alt_x:.1f}, {alt_y:.1f})")
                                        tello.move_left(move_step)  # positive y = left
                                        time.sleep(0.5)  # Delay after movement
                                        ego_y += move_step
                                    else:
                                        print(f"Taking alternative path: Moving right to ({alt_x:.1f}, {alt_y:.1f})")
                                        tello.move_right(move_step)  # negative y = right
                                        time.sleep(0.5)  # Delay after movement
                                        ego_y -= move_step
                                    alternative_path_y_taken = True  # Prohibit y-direction temporarily
                                state_history.append((ego_x, ego_y, ego_z))
                                print(f"Alternative path taken. Current position: ({ego_x:.1f}, {ego_y:.1f}); alternative_path_x_taken is {alternative_path_x_taken}, alternative_path_y_taken is {alternative_path_y_taken}")
                                movement_made = True
                                time.sleep(0.5)
                            else:
                                print(f"No alternative path found for Y movement.")
                    
                    # Execute movement if safe
                    if can_move_y and not alternative_path_y_taken:
                        if dy > 0:
                            tello.move_left(move_step)  # positive y = left
                            time.sleep(0.5)  # Delay after movement
                        else:
                            tello.move_right(move_step)  # negative y = right
                            time.sleep(0.5)  # Delay after movement
                        ego_y = new_y
                        state_history.append((ego_x, ego_y, ego_z))
                        print(f"Can move y. Moving towards goal {goal_index + 1}: ({target_x}, {target_y}), Current: ({ego_x:.1f}, {ego_y:.1f})")
                        movement_made = True
                        time.sleep(0.5)

                #Strategic reset of flags
                if alternative_path_x_taken:
                    if ego_y == target_y:
                        alternative_path_x_taken = False
                        print("Resetting alternative_path_x_taken to False, since we are at the target y coordinate")

                if alternative_path_y_taken:
                    if ego_x == target_x:
                        alternative_path_y_taken = False
                        print("Resetting alternative_path_y_taken to False, since we are at the target x coordinate")


                # If completely stuck (can't move in either direction and no alternative path)
                if not movement_made and adversary_active:
                    print(f"STUCK: Cannot move towards goal {goal_index + 1} from current position ({ego_x}, {ego_y})")
                    print(f"Skipping goal {goal_index + 1} and moving to next goal...")
                    goal_index += 1
                    time.sleep(0.5)
                    
                    # Check if we've processed all goals
                    if goal_index >= len(goals):
                        print("All goals processed! Landing the drone...")
                        if drone_in_air:
                            tello.land()
                            time.sleep(3)  # Delay after landing
                            state_history.append((ego_x, ego_y, ego_z))  # Record final state
                            print("Drone has landed.")
                            print(f"Final state: ({ego_x}, {ego_y}, {ego_z})")
                            print(f"Total states recorded: {len(state_history)}")
                        break

            
            else:
                # Goal reached
                print(f"Goal {goal_index + 1} reached! ({target_x}, {target_y})")
                goal_index += 1
                time.sleep(1)
                
                if goal_index >= len(goals):
                    print("All goals processed! Landing the drone...")
                    if drone_in_air:
                        tello.land()
                        time.sleep(3)  # Delay after landing
                        state_history.append((ego_x, ego_y, ego_z))  # Record final state
                        print("Drone has landed.")
                        print(f"Final state: ({ego_x}, {ego_y}, {ego_z})")
                        print(f"Total states recorded: {len(state_history)}")
                    break




        # Landing control
        if key == ord('q'):
            if drone_in_air:
                tello.land()
                time.sleep(3)  # Delay after landing to allow drone to settle
                state_history.append((ego_x, ego_y, ego_z))  # Record final state before landing
                print("Drone has landed.")
                print(f"Final state: ({ego_x}, {ego_y}, {ego_z})")
                print(f"Total states recorded: {len(state_history)}")
            break



#####################################
# Landing the drone / finish program
#####################################
except Exception as e:
    print(f"Error occurred: {e}")
    if drone_in_air:
        print("Error occurred. Landing the drone.")
        tello.land()
        time.sleep(3)  # Delay after landing

except KeyboardInterrupt:
    print("Keyboard Interrupt received. Landing the drone.")
    if drone_in_air:
        tello.land()
        time.sleep(3)  # Delay after landing
    tello.streamoff()
    cv2.destroyAllWindows()

finally:
    tello.streamoff()
    cv2.destroyAllWindows()


Battery level: 82%
Press 't' to takeoff, 'q' to land and quit.
Drone is now airborne and hovering.
Drone moved up 30 cm. Current state: (0, 0, 20)
Pursuer set at position: (40, 0, 20)
Adversary detected at (40, 0), avoidance radius: 30 cm
Starting next movement cycle, for now alternative_path_x_taken is False and alternative_path_y_taken is False
Can move y. Moving towards goal 1: (0, 80), Current: (0.0, 40.0)
Starting next movement cycle, for now alternative_path_x_taken is False and alternative_path_y_taken is False
Can move y. Moving towards goal 1: (0, 80), Current: (0.0, 80.0)
Goal 1 reached! (0, 80)
Starting next movement cycle, for now alternative_path_x_taken is False and alternative_path_y_taken is False
Can move x. Moving towards goal 2: (40, 80), Current: (40.0, 80.0)
Goal 2 reached! (40, 80)
Starting next movement cycle, for now alternative_path_x_taken is False and alternative_path_y_taken is False
Can move y. Moving towards goal 3: (40, -80), Current: (40.0, 40.0)
Startin